## Load Libraries and Slice Name

In [ ]:
from state import *
from IPython.core.getipython import get_ipython

# Import libraries
%run ../setup/include/include_libraries.py
# Import selected slice name
%run ../slice_info/selected_slice.py
# Import topology_variables
# If you manually run the notebook, please specify the slice name. 
# Use the button in the GUI automatically does it for you
if os.getenv('SELECTED_SLICE') is not None:
    SELECTED_SLICE = os.getenv('SELECTED_SLICE')

# If you manually run the notebook, please specify the slice name in the line below
# SELECTED_SLICE = 'slice_name'
path = f'../slice_info/{SELECTED_SLICE}/topology_variables.ipynb'
get_ipython().run_line_magic('run', path)

In [ ]:
try:
    slice = fablib.get_slice(name=selected_slice)
    '''
    network1 = slice.get_network(name=network1_name)
    network1_available_ips = network1.get_available_ips()
    network1.show()
    
    network2 = slice.get_network(name=network2_name)
    network2_available_ips =  network2.get_available_ips()
    network2.show()
    
    network3 = slice.get_network(name=network3_name)
    network3_available_ips =  network3.get_available_ips()
    network3.show()  
    '''
    success=True
except Exception as e:
    print(f"Exception: {e}")
    success=False
    raise SystemExit("Stopping notebook execution due to exception")

## Configure Node1

In [ ]:
try:
    node1 = slice.get_node(name=node1_name)        
    node1_iface0 = node1.get_interface(network_name=network1_name)  
    node1_iface0.ip_addr_add(addr=node1_IPv4_addr0, subnet=subnet1)
    
    node1_iface1 = node1.get_interface(network_name=network4_name)  
    node1_iface1.ip_addr_add(addr=node1_IPv4_addr1, subnet=subnet4)
    
    '''
    node1.ip_route_add(subnet=network2.get_subnet(), gateway=network1.get_gateway())
    node1.ip_route_add(subnet=network3.get_subnet(), gateway=network1.get_gateway())
    '''
    #stdout, stderr = node1.execute(f'ip addr show {node1_iface.get_os_interface()}')
    stdout, stderr = node1.execute(f'ip route list')
    success=True

except Exception as e:
    print(f"Exception: {e}")
    success=False
    raise SystemExit("Stopping notebook execution due to exception")

## Configure Node2

In [ ]:
try:
    node2 = slice.get_node(name=node2_name)        
    node2_iface0 = node2.get_interface(network_name=network1_name)  
    node2_iface0.ip_addr_add(addr=node2_IPv4_addr0, subnet=subnet1)

    node2_iface1 = node2.get_interface(network_name=network2_name)  
    node2_iface1.ip_addr_add(addr=node2_IPv4_addr1, subnet=subnet2)
    
    '''
    node2.ip_route_add(subnet=network1.get_subnet(), gateway=network2.get_gateway())
    node2.ip_route_add(subnet=network3.get_subnet(), gateway=network2.get_gateway())
    '''
    
    #stdout, stderr = node2.execute(f'ip addr show {node2_iface.get_os_interface()}')    
    stdout, stderr = node2.execute(f'ip route list')
    success=True

except Exception as e:
    print(f"Exception: {e}")
    success=False
    raise SystemExit("Stopping notebook execution due to exception")

## Configure Node3

In [ ]:
try:
    node3 = slice.get_node(name=node3_name)        
    node3_iface0 = node3.get_interface(network_name=network2_name)  
    node3_iface0.ip_addr_add(addr=node3_IPv4_addr0, subnet=subnet2)

    node3_iface1 = node3.get_interface(network_name=network3_name)  
    node3_iface1.ip_addr_add(addr=node3_IPv4_addr1, subnet=subnet3)
    
    #stdout, stderr = node3.execute(f'ip addr show {node3_iface.get_os_interface()}')    
    stdout, stderr = node3.execute(f'ip route list')
    success= True

except Exception as e:
    print(f"Exception: {e}")
    success=False
    raise SystemExit("Stopping notebook execution due to exception")

## Configure Node4

In [ ]:
try:
    node4 = slice.get_node(name=node4_name)        
    node4_iface0 = node4.get_interface(network_name=network3_name)  
    node4_iface0.ip_addr_add(addr=node4_IPv4_addr0, subnet=subnet3)
    
    A_Device_Name = node4_iface0.get_device_name()
    node4_iface1 = node4.get_interface(network_name=network4_name)  
    node4_iface1.ip_addr_add(addr=node4_IPv4_addr1, subnet=subnet4)
    
    B_Device_Name = node4_iface1.get_device_name()
    
    #stdout, stderr = node4.execute(f'ip addr show {node3_iface.get_os_interface()}')    
    stdout, stderr = node4.execute(f'ip route list')
    success=True

except Exception as e:
    print(f"Exception: {e}")
    success=False
    raise SystemExit("Stopping notebook execution due to exception")

## Configure All nodes

In [ ]:
try:
    INTERFACE_NAME_HOLDER1 = "INTERFACE_NAME_1"
    INTERFACE_NAME_HOLDER2 = "INTERFACE_NAME_2"
    
    for node in slice.get_nodes():
        node.execute('sudo DEBIAN_FRONTEND=noninteractive apt-get install -y net-tools')
        node.execute('wget https://github.com/fabric-testbed/teaching-materials/raw/main/assignments/OSPF/scripts/ospf-script.sh')
        
        node.execute(f"sed -i 's/{INTERFACE_NAME_HOLDER1}/{A_Device_Name}/' ospf-script.sh")
        node.execute(f"sed -i 's/{INTERFACE_NAME_HOLDER2}/{B_Device_Name}/' ospf-script.sh")
        node.execute("chmod +x ospf-script.sh")
        node.execute("./ospf-script.sh")
        node.execute("sudo cp /usr/share/doc/quagga-core/examples/vtysh.conf.sample /etc/quagga/vtysh.conf")
        node.execute("sudo cp /usr/share/doc/quagga-core/examples/bgpd.conf.sample /etc/quagga/bgpd.conf")
        node.execute("sudo chown quagga:quagga /etc/quagga/*.conf")
        node.execute("sudo chown quagga:quaggavty /etc/quagga/vtysh.conf")
        node.execute("sudo chmod 640 /etc/quagga/*.conf")
        node.execute("sudo service zebra start")
        node.execute("sudo service ospfd start")
        
        for interface in node.get_interfaces():
            stdout, stderr = node.execute(f'sudo ip link set dev { interface.get_device_name()} up')
        print(f"Network status at node {node.get_name()}")
        node.execute("ip a")
    success=True
        
except Exception as e:
    print(f"Exception: {e}")
    success= False
    raise SystemExit("Stopping notebook execution due to exception")

## Update state

In [ ]:
if success:
    update_state_local(selected_slice, 'SLICE_CONFIGURED')
    update_state_in_fim(selected_slice, read_state_from_local(selected_slice))
else:
    print ('State is not updated due to errors')